# DSCI 417 – Homework 01

**Lauren Forti**

In [0]:
# imports
import math
from pyspark.sql import SparkSession
from pyspark.mllib.random import RandomRDDs

In [0]:
# create objects
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Problem 1: Terminology

1. Scala
2. SparkSession
3. SparkContext
4. Resilient Distributed Dataset
5. Partitions
6. Transformation
7. Action
8. Transformation
9. Action
10. Transformation
11. Action
12. List
13. Master node
14. Workers
15. Driver
16. Executor

## Problem 2: Working with a Numerical RDD

In [0]:
# create RDD w/ 1.2 million elements selected from [0,1]
random_rdd = RandomRDDs.uniformRDD(sc, size=1200000, seed=1)

# get descriptive statistics
print('Sum:     ', random_rdd.sum())
print('Mean:    ', random_rdd.mean())
print('Std Dev: ', random_rdd.stdev())
print('Minimum: ', random_rdd.min())
print('Maximum: ', random_rdd.max())

Sum: 599731.0925959006
Mean: 0.4997759104965818
Std Dev: 0.2887300658502359
Minimum: 1.0351479373671424e-07
Maximum: 0.9999990237273222

In [0]:
# determine the # of partitions
partition_num = random_rdd.getNumPartitions()

# create list w/ # of elements within each of the partitions
partition_list = random_rdd.glom().map(len).collect()
# output list
print('Number of Partitions: ', partition_num)
print('Size of Partitions:\n', partition_list)

Number of Partitions: 8
Size of Partitions:
 [150000, 150000, 150000, 150000, 150000, 150000, 150000, 150000]

## Problem 3: Transformations

In [0]:
# scale random_rdd by 10
scaled_rdd = random_rdd.map(lambda x : x*10)

# get descriptive statistics
print('Sum:     ', scaled_rdd.sum())
print('Mean:    ', scaled_rdd.mean())
print('Std Dev: ', scaled_rdd.stdev())
print('Minimum: ', scaled_rdd.min())
print('Maximum: ', scaled_rdd.max())

Sum: 5997310.925958995
Mean: 4.997759104965838
Std Dev: 2.887300658502357
Minimum: 1.0351479373671424e-06
Maximum: 9.999990237273222

In [0]:
# take the ln of each element
log_rdd = scaled_rdd.map(lambda x : math.log(x))

# get descriptive statistics
print('Sum:     ', log_rdd.sum())
print('Mean:    ', log_rdd.mean())
print('Std Dev: ', log_rdd.stdev())
print('Minimum: ', log_rdd.min())
print('Maximum: ', log_rdd.max())

Sum: 1561638.08183098
Mean: 1.3013650681924729
Std Dev: 1.0021344727804842
Minimum: -13.780966206806882
Maximum: 2.302584116720891

## Problem 4: Calculating SSE

In [0]:
# read data file into RDD
pairs_raw = sc.textFile('/FileStore/tables/pairs_data.txt')
# get # of elements
print(pairs_raw.count())

12743548

In [0]:
# display first 5 elements
for row in pairs_raw.take(5):
  print(row)

12.3 12.1
9.1 8.7
9.3 9.9
8.5 8.5
11.2 10.8

In [0]:
# function to split strings into float tuples
def process_line(row):
  items = row.split(' ')
  return(float(items[0]), float(items[1]))

# call process_line
pairs = pairs_raw.map(process_line)

# print first 5 elements
for row in pairs.take(5):
  print(row)

(12.3, 12.1)
(9.1, 8.7)
(9.3, 9.9)
(8.5, 8.5)
(11.2, 10.8)

In [0]:
# calc SSE
SSE = pairs.map(lambda x : (x[0] - x[1])**2).sum()
#output results
print(f'The value of SSE is {SSE}')

The value of SSE is 4597380.190042952

## Problem 5: Calculating r-Squared

In [0]:
# get the mean of the first element in each tuple
mean = pairs.map(lambda x : x[0]).mean()
# output results
print(f'The mean is {mean}')

The mean is 10.00013136059118

In [0]:
# calc SST
SST = pairs.map(lambda x: (x[0] - mean)**2).sum()
# output results
print(f'The value of SST is {SST}')

The value of SST is 24980514.859974924

In [0]:
# calc r-squared score
r2 = 1 - SSE/SST
# output results
print(f'The r-Squared score is {r2}')

The r-Squared score is 0.815961351644953

## Problem 6: NASA Server Logs

In [0]:
# read data file into RDD
nasa = sc.textFile('/FileStore/tables/NASA_server_logs_Aug_1995.txt')
print(nasa.count())

1569888

In [0]:
# display first 5 elements
for row in nasa.take(5):
  print(row)

in24.inetnebr.com [01/Aug/1995:00:00:01] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt" 200 1839
uplherc.upl.com [01/Aug/1995:00:00:07] "GET /" 304 0
uplherc.upl.com [01/Aug/1995:00:00:08] "GET /images/ksclogo-medium.gif" 304 0
uplherc.upl.com [01/Aug/1995:00:00:08] "GET /images/MOSAIC-logosmall.gif" 304 0
uplherc.upl.com [01/Aug/1995:00:00:08] "GET /images/USA-logosmall.gif" 304 0

In [0]:
# convert each line to Boolean value if contains specific string and count
num_get = nasa.map(lambda x: 'GET' in x).sum()
num_post = nasa.map(lambda x: 'POST' in x).sum()
num_head = nasa.map(lambda x: 'HEAD' in x).sum()

# output results
print(f'Number of GET requests:  {num_get}')
print(f'Number of POST requests: {num_post}')
print(f'Number of HEAD requests: {num_head}')

Number of GET requests: 1565812
Number of POST requests: 111
Number of HEAD requests: 3965